In [4]:
import numpy as np
from scipy.stats import entropy
from scipy.sparse import issparse
from joblib import Parallel, delayed
from scipy.sparse import csr_matrix
import time

def mutual_information_matrix_serial(matrix, nbins=20, n_jobs=-1):
    """
    Computes the mutual information matrix in parallel, working directly with sparse matrices,
    and only computes the upper triangular part of the matrix.
    """
    if not issparse(matrix):
        matrix = csr_matrix(matrix)

    n_features = matrix.shape[0]
    mi_matrix = np.zeros((n_features, n_features))

    def compute_pairwise_mi(vi, vj, nbins=20):
        joint_counts, _, _ = np.histogram2d(vi, vj, bins=nbins)
        ncounts = joint_counts.sum()
        if ncounts == 0:
            return 0  # No mutual information if no overlap
        joint_prob = joint_counts / ncounts

        marginal_i = joint_prob.sum(axis=1) + 1e-10
        marginal_j = joint_prob.sum(axis=0) + 1e-10

        h_xy = entropy(joint_prob.flatten(), base=2)
        h_x = entropy(marginal_i, base=2)
        h_y = entropy(marginal_j, base=2)

        return float(h_x + h_y - h_xy)

    for i in range(n_features):
        for j in range(i, n_features):
            vi = matrix[i, :].toarray().flatten()
            vj = matrix[j, :].toarray().flatten()
            mi_matrix[i, j] = compute_pairwise_mi(vi, vj, nbins=nbins)
            if i != j:
                mi_matrix[j, i] = mi_matrix[i, j]  # Exploit symmetry
    return mi_matrix

def mutual_information_matrix_parallel(matrix, nbins=20, n_jobs=-1):
    """
    Computes the mutual information matrix in parallel, working directly with sparse matrices,
    and computes the full matrix (including the diagonal elements).
    """
    if not issparse(matrix):
        matrix = csr_matrix(matrix)

    n_features = matrix.shape[0]
    mi_matrix = np.zeros((n_features, n_features))

    def compute_pairwise_mi(i, j, matrix, nbins=20):
        """
        Computes mutual information between row i and row j of the sparse matrix.
        """
        vi = matrix[i, :].toarray().flatten()
        vj = matrix[j, :].toarray().flatten()
        
        joint_counts, _, _ = np.histogram2d(vi, vj, bins=nbins)
        ncounts = joint_counts.sum()
        if ncounts == 0:
            return 0  # No mutual information if no overlap
        joint_prob = joint_counts / ncounts

        marginal_i = joint_prob.sum(axis=1) + 1e-10
        marginal_j = joint_prob.sum(axis=0) + 1e-10

        h_xy = entropy(joint_prob.flatten(), base=2)
        h_x = entropy(marginal_i, base=2)
        h_y = entropy(marginal_j, base=2)

        return float(h_x + h_y - h_xy)

    # Parallelizing the pairwise mutual information computation
    jobs = [(i, j) for i in range(n_features) for j in range(i, n_features)]  # Includes diagonal
    results = Parallel(n_jobs=n_jobs)(
        delayed(compute_pairwise_mi)(i, j, matrix, nbins) for i, j in jobs
    )

    # Fill the matrix with the results
    for idx, (i, j) in enumerate(jobs):
        mi_matrix[i, j] = results[idx]
        mi_matrix[j, i] = results[idx]  # Exploit symmetry to avoid duplicate computation

    return mi_matrix

In [5]:
import numpy as np
from scipy.stats import entropy
from joblib import Parallel, delayed
 
def mutual_information_matrix_df(df, bins=20, n_jobs=-1):
    """
    Computes the mutual information matrix using NumPy's vectorized operations and parallel processing.
 
    Args:
        df (pd.DataFrame): Input DataFrame.
        n_jobs (int, optional): Number of jobs for parallel processing. Defaults to 4.
 
    Returns:
        np.ndarray: Mutual information matrix.
    """
 
    n_features = df.shape[1]
    mi_matrix = np.zeros((n_features, n_features))
 
    def compute_mi(i, j):
        joint_prob = np.histogram2d(df.iloc[:, i], df.iloc[:, j], bins=bins)[0]
        joint_prob /= joint_prob.sum() 
 
        marginal_prob_i = joint_prob.sum(axis=1)
        marginal_prob_j = joint_prob.sum(axis=0)
 
        mi = entropy(joint_prob.flatten(), base=2) - entropy(marginal_prob_i,base=2) - entropy(marginal_prob_j,base=2)
        return mi
 
    jobs = [(i, j) for i in range(n_features) for j in range(i + 1, n_features)]
    results = Parallel(n_jobs=n_jobs)(delayed(compute_mi)(*job) for job in jobs)
 
    for (i, j), mi in zip(jobs, results):
        mi_matrix[i, j] = mi
        mi_matrix[j, i] = mi
 
    np.fill_diagonal(mi_matrix, 1)  # Self-information is always 1
 
    return mi_matrix

In [2]:
from scipy.sparse import random as sparse_random
from scipy.io import mmwrite

# Generate a sparse random matrix with 1000 rows and 5000 columns
# Density of the matrix is set to 0.01 (1% non-zero elements)
sparse_matrix = sparse_random(5000, 5000, density=0.01, format='csr')

mmwrite("sparse_matrix.mtx", sparse_matrix)

In [ ]:
import pandas as pd
dense_matrix = sparse_matrix.toarray().T  # Features as columns
df = pd.DataFrame(dense_matrix)

start_time = time.time()

mi_matrix_df = mutual_information_matrix_df(df, bins=20, n_jobs=-1)

end_time = time.time()
elapsed_time = end_time - start_time
print(f"Elapsed time: {elapsed_time:.4f} seconds") 

In [27]:
start_time = time.time()

mi_matrix = mutual_information_matrix_parallel(sparse_matrix, nbins=20, n_jobs=-1)

end_time = time.time()
elapsed_time = end_time - start_time
print(f"Elapsed time: {elapsed_time:.4f} seconds") 


Elapsed time: 3042.1938 seconds


In [28]:
vec1 = [1, 2, 3, 0, 0]  # Row 0
vec2 = [4, 0, 6, 0, 0]  # Row 1
vec3 = [0, 1, 3, 7, 9]  # Row 2
vec4 = [5, 0, 0, 0 ,2] # Row 3
matrix = np.array([vec1, vec2, vec3, vec4])  # Use NumPy array for efficiency


In [29]:
import pandas as pd
dense_matrix = matrix.T
df = pd.DataFrame(dense_matrix)

start_time = time.time()

mi_matrix_df = mutual_information_matrix_df(df, bins=20, n_jobs=-1)

end_time = time.time()
elapsed_time = end_time - start_time
print(f"Elapsed time: {elapsed_time:.4f} seconds") 

Elapsed time: 0.0317 seconds


In [ ]:
start_time = time.time()

mi_matrix = mutual_information_matrix_parallel(matrix, nbins=20, n_jobs=-1)

end_time = time.time()
elapsed_time = end_time - start_time
print(f"Elapsed time: {elapsed_time:.4f} seconds") 

Elapsed time: 0.0316 seconds


In [31]:
mi_matrix[0:4,0:4]

array([[1.9219282 , 1.3709507 , 1.9219282 , 0.9709507 ],
       [1.3709507 , 1.37095071, 1.3709507 , 0.81997321],
       [1.9219282 , 1.3709507 , 2.32192819, 1.3709507 ],
       [0.9709507 , 0.81997321, 1.3709507 , 1.37095071]])

In [32]:
mi_matrix_df[0:4,0:4]

array([[ 1.        , -1.37095059, -1.92192809, -0.97095059],
       [-1.37095059,  1.        , -1.37095059, -0.81997309],
       [-1.92192809, -1.37095059,  1.        , -1.37095059],
       [-0.97095059, -0.81997309, -1.37095059,  1.        ]])

In [ ]:
sparse_matrix_mi = csr_matrix(mi_matrix)
print(sparse_matrix_mi)
mmwrite("sparse_matrix_mi.mtx", sparse_matrix_mi)

In [9]:
start_time = time.time()

mi_matrix = mutual_information_matrix_parallel(sparse_matrix, nbins=20, n_jobs=-1)

end_time = time.time()
elapsed_time = end_time - start_time
print(f"Elapsed time: {elapsed_time:.4f} seconds") 

sparse_matrix_mi = csr_matrix(mi_matrix)
print(sparse_matrix_mi)
mmwrite("sparse_matrix_mi.mtx", sparse_matrix_mi)

Elapsed time: 4365.2635 seconds
<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 25000000 stored elements and shape (5000, 5000)>
  Coords	Values
  (0, 0)	0.08595718539621552
  (0, 1)	8.695767045516223e-05
  (0, 2)	0.0001009282230546582
  (0, 3)	0.0021832049015041155
  (0, 4)	0.00010265814458118583
  (0, 5)	9.906287545818904e-05
  (0, 6)	0.00010296923787081469
  (0, 7)	0.00012718322926646985
  (0, 8)	0.00011097445947733098
  (0, 9)	0.00010094093701806806
  (0, 10)	0.0001068829646203584
  (0, 11)	9.042029584752087e-05
  (0, 12)	0.00011710334953562995
  (0, 13)	0.00012103499616814006
  (0, 14)	0.0017203431515214473
  (0, 15)	0.00011506174719264073
  (0, 16)	8.070379777949666e-05
  (0, 17)	0.0013403142832507375
  (0, 18)	7.867074161838072e-05
  (0, 19)	0.00011288742424414577
  (0, 20)	9.669157042743737e-05
  (0, 21)	8.228068031501667e-05
  (0, 22)	0.00011694402415221572
  (0, 23)	0.00012938554579056127
  (0, 24)	0.00013740750799048906
  :	:
  (4999, 4975)	0.000106136075548546

In [6]:
print(mi_matrix.shape)

(500, 500)


In [1]:
# Enhanced C++ integration with file I/O
import subprocess
import os
from scipy.io import mmread, mmwrite

def compute_mi_cpp_optimized(sparse_matrix, method="openmp", num_threads=None):
    """
    Compute mutual information using C++ implementation and return results to Python
    
    Args:
        sparse_matrix: Input sparse matrix
        method: "openmp" or "serial"
        num_threads: Number of threads for OpenMP (None for auto)
    
    Returns:
        MI matrix as numpy array
    """
    # Save input matrix
    mmwrite("sparse_matrix.mtx", sparse_matrix)
    
    # Choose executable
    executable = f"./mi_{method}"
    if not os.path.exists(executable):
        raise FileNotFoundError(f"C++ executable {executable} not found. Please compile first.")
    
    # Run C++ program
    cmd = [executable]
    if method == "openmp" and num_threads:
        cmd.append(str(num_threads))
    
    print(f"Running C++ computation with {method} method...")
    start_time = time.time()
    
    result = subprocess.run(cmd, capture_output=True, text=True)
    
    if result.returncode != 0:
        raise RuntimeError(f"C++ execution failed: {result.stderr}")
    
    end_time = time.time()
    print(f"C++ execution time: {end_time - start_time:.4f} seconds")
    print("C++ output:", result.stdout)
    
    # Read results
    if os.path.exists("sparse_matrix_mi.mtx"):
        mi_sparse = mmread("sparse_matrix_mi.mtx")
        return mi_sparse.toarray()
    else:
        raise FileNotFoundError("C++ did not generate output file")

# Example usage
# mi_result = compute_mi_cpp_optimized(sparse_matrix, method="openmp", num_threads=8)

In [ ]:
# Binary format for faster I/O
import struct

def write_binary_matrix(matrix, filename):
    """Write matrix in binary format for C++ consumption"""
    if issparse(matrix):
        matrix = matrix.toarray()
    
    rows, cols = matrix.shape
    with open(filename, 'wb') as f:
        # Write header: rows, cols
        f.write(struct.pack('ii', rows, cols))
        # Write data as doubles
        matrix.astype(np.float64).tobytes()
        f.write(matrix.tobytes())

def read_binary_matrix(filename):
    """Read binary matrix from C++ output"""
    with open(filename, 'rb') as f:
        # Read header
        rows, cols = struct.unpack('ii', f.read(8))
        # Read data
        data = f.read(rows * cols * 8)  # 8 bytes per double
        matrix = np.frombuffer(data, dtype=np.float64)
        return matrix.reshape(rows, cols)

def compute_mi_cpp_binary(sparse_matrix, method="openmp", num_threads=None):
    """Fast binary I/O version"""
    # Write input in binary format
    write_binary_matrix(sparse_matrix, "input_matrix.bin")
    
    # Run C++ (would need to modify C++ to handle binary I/O)
    executable = f"./mi_{method}_binary"  # Modified version
    cmd = [executable]
    if method == "openmp" and num_threads:
        cmd.append(str(num_threads))
    
    result = subprocess.run(cmd, capture_output=True, text=True)
    
    if result.returncode != 0:
        raise RuntimeError(f"C++ execution failed: {result.stderr}")
    
    # Read binary output
    return read_binary_matrix("output_mi.bin")

In [ ]:
# Option 3: Python C Extension approach
# This would require creating a pybind11 wrapper

# Example of how it would look (after creating the extension):

import py_mi_openmp  # Compiled C++ extension

def compute_mi_native(sparse_matrix, method="openmp", num_threads=8, nbins=20):
    '''
    Direct C++ function call from Python - no file I/O overhead
    '''
    if issparse(sparse_matrix):
        # Convert to format expected by C++
        matrix_array = sparse_matrix.toarray()
    else:
        matrix_array = sparse_matrix
    
    # Direct function call to C++ implementation
    result = py_mi_openmp.run(
        matrix_array, 
        nbins=nbins, 
        num_threads=num_threads,
        use_openmp=(method=="openmp")
    )
    
    return result

# Usage would be:
mi_result = compute_mi_native(sparse_matrix, method="openmp", num_threads=8)

print("pybind11 extension would eliminate file I/O overhead completely")

In [ ]:
# Option 4: Shared Memory approach
import mmap
import tempfile

def compute_mi_shared_memory(sparse_matrix, method="openmp", num_threads=8):
    """
    Use shared memory for zero-copy data transfer
    """
    if issparse(sparse_matrix):
        matrix = sparse_matrix.toarray()
    else:
        matrix = np.array(sparse_matrix)
    
    rows, cols = matrix.shape
    
    # Create shared memory files
    with tempfile.NamedTemporaryFile(delete=False) as input_file:
        input_name = input_file.name
        # Write matrix to shared memory
        matrix.astype(np.float64).tofile(input_file)
    
    with tempfile.NamedTemporaryFile(delete=False) as output_file:
        output_name = output_file.name
    
    # Run C++ with shared memory file paths
    cmd = [f"./mi_{method}_shared", input_name, output_name, str(rows), str(cols)]
    if num_threads:
        cmd.append(str(num_threads))
    
    result = subprocess.run(cmd, capture_output=True, text=True)
    
    if result.returncode != 0:
        raise RuntimeError(f"C++ execution failed: {result.stderr}")
    
    # Read results from shared memory
    result_matrix = np.fromfile(output_name, dtype=np.float64).reshape(rows, rows)
    
    # Cleanup
    os.unlink(input_name)
    os.unlink(output_name)
    
    return result_matrix

In [8]:
# Performance comparison framework
def benchmark_all_methods(sparse_matrix, num_threads=16):
    """
    Compare all mutual information computation methods
    """
    results = {}
    
    print("Benchmarking different MI computation methods...")
    print(f"Matrix size: {sparse_matrix.shape}")
    print(f"Density: {sparse_matrix.nnz / (sparse_matrix.shape[0] * sparse_matrix.shape[1]):.4f}")
    print("-" * 50)
    
    # # 1. Pure Python (your original)
    # print("1. Testing Python parallel implementation...")
    # start_time = time.time()
    # mi_python = mutual_information_matrix_parallel(sparse_matrix, nbins=20, n_jobs=num_threads)
    python_time = 3042.1938
    # results['python_parallel'] = {'time': python_time, 'result': mi_python}
    # print(f"   Time: {python_time:.4f} seconds")
    
    # 2. C++ via file I/O (if compiled)
    try:
        print("2. Testing C++ OpenMP with file I/O...")
        start_time = time.time()
        mi_cpp = compute_mi_cpp_optimized(sparse_matrix, method="openmp", num_threads=num_threads)
        cpp_time = time.time() - start_time
        results['cpp_openmp'] = {'time': cpp_time, 'result': mi_cpp}
        print(f"   Time: {cpp_time:.4f} seconds")
        
        # Calculate speedup
        speedup = python_time / cpp_time
        print(f"   Speedup over Python: {speedup:.2f}x")
        
        # # Calculate accuracy
        # error = np.sqrt(np.mean((mi_python - mi_cpp)**2))
        # print(f"   RMSE vs Python: {error:.6f}")
        
    except Exception as e:
        print(f"   C++ method failed: {e}")
    
    # # 3. DataFrame method
    # print("3. Testing DataFrame method...")
    # dense_matrix = sparse_matrix.toarray().T
    # df = pd.DataFrame(dense_matrix)
    # start_time = time.time()
    # mi_df = mutual_information_matrix_df(df, bins=20, n_jobs=num_threads)
    # df_time = time.time() - start_time
    # results['dataframe'] = {'time': df_time, 'result': mi_df}
    # print(f"   Time: {df_time:.4f} seconds")
    
    return results

# Test with your current matrix
print("Ready to benchmark - uncomment the line below to run:")
benchmark_results = benchmark_all_methods(sparse_matrix, num_threads=8)

Ready to benchmark - uncomment the line below to run:
Benchmarking different MI computation methods...
Matrix size: (5000, 5000)
Density: 0.0100
--------------------------------------------------
2. Testing C++ OpenMP with file I/O...
Running C++ computation with openmp method...
C++ execution time: 521.2266 seconds
C++ output: Requested procs : 8
Max procs available: 16
Time taken for the parallel for loop: 513.12 seconds

   Time: 522.1245 seconds
   Speedup over Python: 5.83x
C++ execution time: 521.2266 seconds
C++ output: Requested procs : 8
Max procs available: 16
Time taken for the parallel for loop: 513.12 seconds

   Time: 522.1245 seconds
   Speedup over Python: 5.83x


In [ ]:
import py_mi_openmp as cpp_imp

cpp_imp.run()
print(cpp_imp.add(2, 4))